# Basic usage of Ollama client

Anton Antonov   
January 2026

----

## Introduction

This notebook shows the basic functionalities of the Raku package ["WWW::Ollama"](https://raku.land/zef:antononcube/WWW::Ollama), [AAp1].

Ollama is a locally deployed Large Language Models (LLMs) runner. (In other words, Ollama allows the download and run of LLMs on users' computers, without relying on cloud-hosted services.)

**Remark:** (Maybe) "Ollama" stands for Omni-Layer Learning Language Acquisition Model.

### Design and implementation details

Here is a list of design and implementation detail items:

- The Ollama access is done via a client object, `WWW::Ollama::Client`
- It is not necessary to use a `WWW::Ollama::Client` object explicitly
- Instead the following functions / subs can be used:
    - `ollama-base-url`
    - `ollama-list-models`
    - `ollama-model-info`
    - `ollama-embedding`
    - `ollama-completion`
    - `ollama-chat-completion`
    - `ollama-client` ("umbrella" function for all of the above)
- The Ollama functions -- and client methods -- take the named option "format"
    - With `format => 'hash'` more details of the request are obtained in Raku hashmap form
- The Ollama host and port can be explicitly specified when a new `WWW::Ollama::Client` object is created
- The method `WWW::Ollama::Client.new` takes the Boolean option (adverb) `:ensure-running`
    - Which is set to false by default, i.e. `:!ensure-running`

This diagram summarizes the Ollama client interaction (in this notebook):


```mermaid
flowchart LR

    Gemma1b(gemma3:1b)
    Gemma12b(gemma3:11b)
    Gemma27b(gemma3:27b)
    Lamma4(lamma4)
    Etc("...")

    OllamaRepo[(Ollama<br>LLM Repository)]
    Ollama{{Ollama<br>executable}}

    OClient("Ollama client<br>(Raku)")

    Req[LLM request]
    
    LLMAval{LLM locally<br>available?}

    Download[[Download LLM]]

    Req --> OClient --> Ollama --> LLMAval 
    LLMAval --> |No| Download
    Download <-.-> OllamaRepo

    NB>notebook]

    NB --> Req
    Ollama -.- Gemma1b
    Ollama -.- Gemma12b
    Ollama -.- Gemma27b
    Ollama -.- Lamma4

    subgraph LocalLLMs["Locally stored LLMs"]
        Gemma1b
        Gemma12b
        Gemma27b
        Lamma4
        Etc
    end   

    IsRunning{"Is Ollama running?"}
    EnsureVal{"ensure-running"}

    IsRunning --> |No|EnsureVal --> |Yes|StartOllama[[Start Ollama executable]]

    StartOllama -.-> Ollama

    Download -.-> LocalLLMs
    subgraph Raku["Raku session"]
        OClient
        Req
        subgraph Ensure["Ensure running"]
            IsRunning
            EnsureVal
            StartOllama
        end
        OClient <--> Ensure
    end
```

---

## Setup

Load Ollama packages and create a new Ollama client object: 

In [20]:
use WWW::Ollama;
use WWW::Ollama::Client;

#my $client = WWW::Ollama::Client.new(host => 'localhost', :11434port);
my $client = WWW::Ollama::Client.new(:!ensure-running);

WWW::Ollama::Client(:base-url("http://127.0.0.1:11434"), :ollama-is-running(Bool::True), :version("0.13.5"), :models-in-memory(0), :local-models(11))

Display packages:

In [ ]:
use Javascript::Google::Charts;
use Data::Translators;
use Data::TypeSystem;

---

## Models

Get the Ollama client _local_ models:

In [2]:
my @models = |ollama-list-models(:$client);

deduce-type(@models)

Vector(Struct([details, digest, model, modified_at, name, size], [Hash, Str, Str, Str, Str, Int]), 11)

Display the model data in a tabular form:

In [3]:
#% html
@models
==> { .sort(*<modified_at>.DateTime ) }()
==> { .map({ $_<modified_at> = $_<modified_at>.DateTime.Str.split('T').head; $_ }) }()
==> { .map({ $_<size-short> = round($_<size> / 1024 ** 3, 0.1) ~ ' GB'; $_ }) }()
==> to-html(field-names => <name size size-short modified_at>)

name,size,size-short,modified_at
qwen3:4b-q4_K_M,2620788260,2.4 GB,2025-09-13
gemma3n:latest,7547589116,7 GB,2025-09-13
gemma3:4b,3338801804,3.1 GB,2026-01-05
gemma3:1b,815319791,0.8 GB,2026-01-05
gemma3:4b-it-qat,4006630865,3.7 GB,2026-01-05
gemma3:12b,8149190253,7.6 GB,2026-01-05
gemma3:27b,17396936941,16.2 GB,2026-01-06
deepseek-r1:latest,5225376047,4.9 GB,2026-01-07
qwen3:4b,2497293931,2.3 GB,2026-01-09
nomic-embed-text:latest,274302450,0.3 GB,2026-01-10


---

## Completion

Here is a text completion:

In [17]:
my $ans = ollama-completion("How many people live in Braz", :$client, format => 'hash');

deduce-type($ans)

Struct([content, durations, finish-reason, model, role, throughput, timestamp, tool-requests, usage], [Str, Hash, Str, Str, Str, Rat, DateTime, Any, Hash])

Here is the text part of the obtained answer is rendered as Markdown:

In [18]:
#% markdown
$ans<content>

As of December 2023, the population of Brazil is approximately **217.6 million** people.

Here's a breakdown of the population by region (estimated):

*   **São Paulo:** 22.6 million
*   **Rio de Janeiro:** 11.9 million
*   **Brasília:** 7.9 million
*   **Amazonas:** 14.7 million
*   **Florida:** 13.9 million
*   **Minas Gerais:** 6.3 million
*   **Santa Catarina:** 4.6 million
*   **Pará:** 13.6 million
*   **Bahia:** 13.3 million
*   **Rio Grande do Sul:** 12.5 million

**Total: Approximately 217.6 million**

You can find more detailed and updated figures on the following websites:

*   **Worldometer:** [https://www.worldometers.info/world-population/brazil-population/](https://www.worldometers.info/world-population/brazil-population/)
*   **United Nations:** [https://www.un.org/development/population/brazil/](https://www.un.org/development/population/brazil/)

Show the rest of the result:

In [19]:
#% html
$ans.grep(*.key ne 'content')
==> to-html

finish-reason stop throughput 89.33172476 role assistant tool-requests (Any) model gemma3:1b usage prompt 15 completion 281 durations eval 2.966400461 load 0.17716825 total 3.3134925 prompt_eval 0.067607917 timestamp 2026-01-19T09:19:03.269421-05:00

---

## Chat

Here is a chat completion:

In [7]:
my $chat-ans = ollama-chat-completion( {role => 'user', content => "How many people live in different states of India?"}, model => 'gemma3:27b', :$client);

deduce-type($chat-ans);

Atom((Str))

Here is the obtained answer is rendered as Markdown:

In [8]:
#% markdown
$chat-ans

Okay, let's break down the population of each state in India as of recent estimates (primarily based on the 2011 Census, with updated projections as of 2023/2024).  Keep in mind these are *estimates* and the actual numbers will change. The last full census was in 2011, and the next one is long overdue (expected in 2024, but may be delayed further).  I'll present the data in a table for clarity.  I will also include the Union Territories.

**Important Notes:**

*   **Data Source:** The primary source is the 2011 Census of India.  More recent population estimates are derived from projections by the Registrar General & Census Commissioner of India, and other demographic studies. These are subject to revision.
*   **Rounding:** Numbers are rounded for readability.
*   **Order:** States are listed alphabetically.
*   **Figures are approximate.**



**Population of Indian States & Union Territories (Approximate - 2024 Estimates)**

| **State/UT**          | **Approximate Population (in Crores)** | **Approximate Population (in Millions)** |
|-----------------------|----------------------------------------|-----------------------------------------|
| Andhra Pradesh        | 8.03                                  | 80.3 million                           |
| Arunachal Pradesh     | 0.14                                  | 1.4 million                            |
| Assam                 | 3.49                                  | 34.9 million                           |
| Bihar                 | 12.34                                 | 123.4 million                          |
| Chandigarh            | 0.11                                  | 1.1 million                            |
| Chhattisgarh          | 3.30                                  | 33.0 million                           |
| Dadra & Nagar Haveli and Daman & Diu | 0.06                               | 0.6 million                            |
| Delhi                 | 1.95                                  | 19.5 million                           |
| Goa                   | 0.15                                  | 1.5 million                            |
| Gujarat               | 6.96                                  | 69.6 million                           |
| Haryana               | 3.11                                  | 31.1 million                           |
| Himachal Pradesh      | 0.70                                  | 7.0 million                            |
| Jammu & Kashmir       | 1.38                                  | 13.8 million                           |
| Jharkhand             | 3.31                                  | 33.1 million                           |
| Karnataka             | 7.60                                  | 76.0 million                           |
| Kerala                | 3.57                                  | 35.7 million                           |
| Ladakh                | 0.03                                  | 0.3 million                            |
| Lakshadweep           | 0.006                                 | 0.06 million                           |
| Madhya Pradesh        | 8.69                                  | 86.9 million                           |
| Maharashtra           | 12.95                                 | 129.5 million                          |
| Manipur               | 0.30                                  | 3.0 million                            |
| Meghalaya             | 0.32                                  | 3.2 million                            |
| Mizoram               | 0.11                                  | 1.1 million                            |
| Nagaland              | 0.19                                  | 1.9 million                            |
| Odisha                | 4.52                                  | 45.2 million                           |
| Puducherry            | 0.15                                  | 1.5 million                            |
| Punjab                | 3.04                                  | 30.4 million                           |
| Rajasthan             | 8.24                                  | 82.4 million                           |
| Sikkim                | 0.07                                  | 0.7 million                            |
| Tamil Nadu            | 8.67                                  | 86.7 million                           |
| Telangana             | 4.06                                  | 40.6 million                           |
| Tripura               | 0.42                                  | 4.2 million                            |
| Uttar Pradesh         | 24.02                                 | 240.2 million                          |
| Uttarakhand           | 1.15                                  | 11.5 million                           |
| West Bengal           | 9.96                                  | 99.6 million                           |
| Andaman and Nicobar Islands | 0.04                               | 0.4 million                            |

**Key Takeaways:**

*   **Most Populous:** Uttar Pradesh is the most populous state, with over 240 million people.
*   **Second Most Populous:** Maharashtra is the second most populous state with close to 130 million people.
*   **Least Populous:**  Ladakh and Lakshadweep are the least populous states/UTs.
*   **Southern India:**  Tamil Nadu, Karnataka, Andhra Pradesh, and Telangana together account for a significant portion of the population.
*   **Eastern India:** Bihar, West Bengal, and Odisha have large populations.




**Where to Find More Detailed/Updated Information:**

*   **Registrar General & Census Commissioner of India:** [https://censusindia.gov.in/](https://censusindia.gov.in/) (This is the official source, but may not have the very latest projections.)
*   **Worldometer (based on UN data):** [https://www.worldometers.info/world-population/india-population/](https://www.worldometers.info/world-population/india-population/) (Provides real-time estimates.)
*   **Statista:** [https://www.statista.com/statistics/1109318/india-population-by-state/](https://www.statista.com/statistics/1109318/india-population-by-state/) (Offers data visualization and analysis)



I hope this comprehensive breakdown is helpful! Let me know if you have any other questions.

---

## Embeddings

Here is a LLM-embedding request of three sentences:

In [9]:
my %body =
        model => 'embeddinggemma',
        input => ["What is the distance from Earth to Mars?", "Distance to the Moon?", "How far is the Sun?"],
        ;

my $ans = $client.embedding(%body);

deduce-type($ans)

Struct([embeddings, model, timestamp, usage], [Array, Str, DateTime, Hash])

Here are the vectors lengths:

In [10]:
say "Number of vectors : {$ans<embeddings>.elems}";
say "Vectors lengths   : {$ans<embeddings>».elems}";

Number of vectors : 3
Vectors lengths   : 768 768 768


----

## LLM Configuration

In order to use Ollama in Raku's LLM-functions framework, [AAp2, AAp3], an LLM-configuration object has to be created:

In [11]:
my $conf = LLM::Functions::Configuration.new(
        name => 'ollama',
        api-key => Whatever,
        api-user-id => 'user:' ~ ((10 ** 11 + 1) .. 10 ** 12).pick,
        module => 'WWW::Ollama',
        base-url => ollama-base-url,
        model => 'gemma3:12b',
        function => &ollama-chat-completion,
        embedding-model => 'nomic-embed-text',
        embedding-function => &ollama-embedding,
        temperature => 0.4,
        max-tokens => 8192,
        total-probability-cutoff => 0,
        prompts => Empty,
        prompt-delimiter => ' ',
        examples => Empty,
        stop-tokens => Empty,
        argument-renames => %(),
        format => 'values');

$conf.Hash.elems

26

### Generating Mermaid-JS code

Generate and _render_ a [Mermaid-JS](https://mermaid.js.org) pie-chart:

In [12]:
#%markdown
my $res = llm-synthesize([
    'How many people live in the different states of India?',
    llm-prompt('NothingElse')('Mermaid-JS pie chart code')
    ],
    e => $conf
);

$res

```mermaid
pie
    "Uttar Pradesh" : 250000000
    "Maharashtra" : 112000000
    "Bihar" : 105000000
    "West Bengal" : 93000000
    "Rajasthan" : 68000000
    "Karnataka" : 61000000
    "Madhya Pradesh" : 60000000
    "Tamil Nadu" : 60000000
    "Telangana" : 39000000
    "Gujarat" : 32000000
    "Kerala" : 33000000
    "Punjab" : 29000000
    "Andhra Pradesh" : 49000000
    "Odisha" : 43000000
    "Haryana" : 29000000
    "Assam" : 31000000
    "Chhattisgarh" : 27000000
    "Goa" : 1400000
    "Manipur" : 2800000
    "Sikkim" : 600000
    "Nagaland" : 2000000
    "Mizoram" : 1100000
    "Tripura" : 3700000
    "Meghalaya" : 3000000
    "Arunachal Pradesh" : 1300000
    "Jharkhand" : 33000000
    "Himachal Pradesh" : 3500000
    "Uttarakhand" : 12000000
```

### Using Google Charts

LLM-generate _computational_ data of state populations of India:

In [13]:
my $res = llm-synthesize([
    'How many people live in the different states of India>',
    llm-prompt('NothingElse')('JSON')
    ],
    e => $conf,
    form => sub-parser('JSON'):drop
);

deduce-type($res)

Vector(Pair(Atom((Str)), Atom((Int))), 29)

Plot using "JavaScript::Google::Charts":

In [14]:
#%html
my $titleTextStyle = { color => 'Ivory' };
my $hAxis = { title => 'population', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => True, format => 'scientific'};
my $vAxis = { title => 'state', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False};
my $chartArea = {left => 150, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

js-google-charts('BarChart', 
    $res.sort(-*.value), 
    title => 'Populations of India states',
    backgroundColor => '#1f1f1f', 
    :$titleTextStyle,
    :$hAxis,
    :$vAxis,
    :horizontal,
    :600height,
    :$chartArea,
    format => 'html', 
    :png-button
);

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

----

## References

[AAp1] Anton Antonov, [WWW::Ollama, Raku package](https://github.com/antononcube/Raku-WWW-Ollama), (2026), [GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov, [LLM::Functions, Raku package](https://github.com/antononcube/Raku-LLM-Functions), (2023-2025), [GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov, [LLM::Prompts, Raku package](https://github.com/antononcube/Raku-LLM-Prompts), (2023-2025), [GitHub/antononcube](https://github.com/antononcube).